In [4]:
import numpy as np
from scipy.stats import norm
import numpy.matlib
import matplotlib.pyplot as plt
import math
import random

In [5]:
#Market Information
risk_free = 0.08

#Share Price Specific Information
S_0 = 100
sigma = 0.3

#call option specific information
K = 100  #The option is struck at-the-money
T=1
L = 150

#Firm specific information
V_0 = 200
sigma_firm = 0.25
debt = 175
recovery_rate = 0.25
corr = 0.2

In [6]:
#Setting our seed before generating the random variables
np.random.seed(0)


dT = 1/12.0

#Function for later valuation

def terminal_value(s_0, risk_free_rate, sigma, z,T): #applies to both firm and stock
    #Generate the terminal share price given some random normal values, z
    return s_0*np.exp((risk_free_rate-sigma**2/2)*T+sigma*np.sqrt(T)*z)
    
#define call pay off function    
def call_payoff(S_T, K):
    #Functions for evaluating the call PRICE in Monte Carlo Estimation
    #The call_payoff function finds the payoff for a call option at terminal time.
    return np.maximum(S_T-K,0)

#share path Function
def price_path(s_0, risk_free_rate, sigma, z, dT):

#Generate the terminal share price given some random normal values, z
    return s_0*np.exp(np.cumsum((risk_free_rate-sigma**2/2)*dT+sigma*np.sqrt(dT)*z,1))

# Adjusted functionto calculate payoff of up and out call option
def up_and_out_call_payoff( sp, K, barrier):
    call_v = [None]*len(sp)
    for x in range(len(sp)):
        if sp[x]<barrier:
            call_v[x] = np.maximum(sp[x]-K,0)
        else:
            call_v[x] = 0
    return call_v

In [7]:
#create empty vectors that we can use to store the value of the Monte Carlo estimates for various sample sizes
#creates an array of the correlations we are going to be testing.
#the array created – it has numbers from −1 to 1, going up in steps of 0.2.
corr_tested = np.linspace(-1,1,11)
corr_tested
#Empty arrays which can be used to store estimate values for different sample sizes
#For each correlation we test, we will have a CVA estimate and a CVA standard deviation.
cva_estimates = [None]*len(corr_tested)
cva_std=[None]*len(corr_tested)
call_value_estimate = [None]*len(corr_tested)
mean_stock_price_per_corr = [None]*len(corr_tested)
mean_firm_value_per_corr = [None]*len(corr_tested)
mean_stock_price_path_per_sample_size = [None]*50
mean_firm_value_path_per_sample_size= [None]*50
call_val_per_sample_size_for_all_correlation = [None]*50
cva_per_sample_size_for_all_correlation = [None]*50

n_simulation = 50 # 50 simulations per month 1000, 2000, ... 50000 sample sizes


In [53]:
#PART 1
for k in range(1,51):
    for i in range(len(corr_tested)):
        correlation = corr_tested[i]
        if (correlation ==1 or correlation ==-1):
            norm_vec_0 = norm.rvs(size=k*1000)   #Generate 50 000 standard normal random variables,
            norm_vec_1 = correlation*norm_vec_0 #Take the vector of these standard normals and multiply them by the correlation
            corr_norm_matrix = np.array([norm_vec_0, norm_vec_1]) #Combine these two vectors into a matrix with each array 
                #making up a row
#If the correlation lies between −0.9 and 0.9 inclusive, we generate the correlated standard normal numbers using a Cholesky decomposition.
        else:
            corr_matrix = np.array([[1, correlation], [correlation, 1]])
            norm_matrix = norm.rvs(size = np.array([2,k*1000]))
            corr_norm_matrix = np.matmul(np.linalg.cholesky(corr_matrix), norm_matrix)
                #simulating the share price(sp) path
        stock_price_path_per_corr = price_path(S_0, risk_free, sigma, corr_norm_matrix[0,], dT)
        mean_stock_price_per_corr[i] = np.mean(stock_price_path_per_corr)
                #simulating the firm value path
        firm_value_path_per_corr = price_path(V_0,risk_free, sigma_firm,corr_norm_matrix [1,], dT)
        mean_firm_value_per_corr[i]=np.mean(firm_value_path_per_corr)
            
    mean_stock_price_path_per_sample_size[k-1]= mean_stock_price_per_corr
    mean_firm_value_path_per_sample_size[k-1] = mean_firm_value_per_corr 


ValueError: axis(=1) out of bounds

range(len(T))

In [ ]:
#2 monte carlo estimates of default free value

In [8]:
#PART 2 monte carlo estimates of default free value
for k in range(1,51):
        for i in range(len(corr_tested)):
            correlation = corr_tested[i]
            if (correlation ==1 or correlation ==-1):
                norm_vec_0 = norm.rvs(size=k*1000)   #Generate 50 000 standard normal random variables,
                norm_vec_1 = correlation*norm_vec_0 #Take the vector of these standard normals and multiply them by the correlation
                corr_norm_matrix = np.array([norm_vec_0, norm_vec_1]) #Combine these two vectors into a matrix with each array 
                #making up a row
#If the correlation lies between −0.9 and 0.9 inclusive, we generate the correlated standard normal numbers using a Cholesky decomposition.
            else:
                corr_matrix = np.array([[1, correlation], [correlation, 1]])
                norm_matrix = norm.rvs(size = np.array([2,k*1000]))
                corr_norm_matrix = np.matmul(np.linalg.cholesky(corr_matrix), norm_matrix)
                #simulating the share price(sp) path
                          
            stock_price = terminal_value(S_0, risk_free, sigma, corr_norm_matrix[0,], 1)
            call_val = up_and_out_call_payoff( stock_price, K, L) #call value for each correlation
            call_value_estimate[i] = np.mean(call_val)
                #simulating the firm value for all correlation per sammple size
            firm_value = terminal_value(V_0, risk_free, sigma_firm, corr_norm_matrix[1,], 1)
            #Amount Lost for all correlation per sample size
            amount_lost = np.exp(-risk_free*1)*(1-recovery_rate)*(firm_value < debt)*call_val
            #CVA FOR ALL CORRELATIONS PER SAMPLE SIZE
            cva_estimates[i] = np.mean(amount_lost)
        call_val_per_sample_size_for_all_correlation[k-1] =call_value_estimate 
        cva_per_sample_size_for_all_correlation[k-1] = cva_estimates
            
     


In [10]:
print(cva_per_sample_size_for_all_correlation)

[[3.2573413020163966, 2.5888036987490155, 2.2202045074633272, 1.9308536816742259, 1.6977735834125787, 1.4154680590369995, 1.1361602251919904, 0.83167682510775998, 0.50531655636149353, 0.15168383664833193, 0.0], [3.2573413020163966, 2.5888036987490155, 2.2202045074633272, 1.9308536816742259, 1.6977735834125787, 1.4154680590369995, 1.1361602251919904, 0.83167682510775998, 0.50531655636149353, 0.15168383664833193, 0.0], [3.2573413020163966, 2.5888036987490155, 2.2202045074633272, 1.9308536816742259, 1.6977735834125787, 1.4154680590369995, 1.1361602251919904, 0.83167682510775998, 0.50531655636149353, 0.15168383664833193, 0.0], [3.2573413020163966, 2.5888036987490155, 2.2202045074633272, 1.9308536816742259, 1.6977735834125787, 1.4154680590369995, 1.1361602251919904, 0.83167682510775998, 0.50531655636149353, 0.15168383664833193, 0.0], [3.2573413020163966, 2.5888036987490155, 2.2202045074633272, 1.9308536816742259, 1.6977735834125787, 1.4154680590369995, 1.1361602251919904, 0.8316768251077599

In [9]:
print( call_val_per_sample_size_for_all_correlation)

[[8.8484748426288515, 8.9000839666024021, 8.9880636662304862, 8.8465254578144616, 8.8542454905372292, 8.9104573848745545, 8.9854141763942774, 8.889184533956449, 9.0137538641912922, 8.9764932839199751, 8.8849453240175844], [8.8484748426288515, 8.9000839666024021, 8.9880636662304862, 8.8465254578144616, 8.8542454905372292, 8.9104573848745545, 8.9854141763942774, 8.889184533956449, 9.0137538641912922, 8.9764932839199751, 8.8849453240175844], [8.8484748426288515, 8.9000839666024021, 8.9880636662304862, 8.8465254578144616, 8.8542454905372292, 8.9104573848745545, 8.9854141763942774, 8.889184533956449, 9.0137538641912922, 8.9764932839199751, 8.8849453240175844], [8.8484748426288515, 8.9000839666024021, 8.9880636662304862, 8.8465254578144616, 8.8542454905372292, 8.9104573848745545, 8.9854141763942774, 8.889184533956449, 9.0137538641912922, 8.9764932839199751, 8.8849453240175844], [8.8484748426288515, 8.9000839666024021, 8.9880636662304862, 8.8465254578144616, 8.8542454905372292, 8.910457384874